In [2]:
import base64
import io
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import cv2
import numpy as np
from dash.dependencies import Input, Output
from PIL import Image

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Canny Edge Detection  Sliders"),
    
    # Image upload
    dcc.Upload(
        id='upload-image',
        children=html.Button('Upload Image'),
        multiple=False
    ),
    
    # Sliders for blur kernel size and Canny thresholds
    dcc.Slider(
        id='kernel-size-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Kernel Size for Blur"),
    
    dcc.Slider(
        id='lower-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=100,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Lower Threshold for Canny Edge Detection"),
    
    dcc.Slider(
        id='upper-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=200,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Upper Threshold for Canny Edge Detection"),
    
    # Graph for displaying the original image and edge detection results side by side
    dcc.Graph(id='edge-detection-graph')
])

# Callback to update edge detection results based on sliders
@app.callback(
    Output('edge-detection-graph', 'figure'),
    [Input('kernel-size-slider', 'value'),
     Input('lower-threshold-slider', 'value'),
     Input('upper-threshold-slider', 'value'),
     Input('upload-image', 'contents')]
)
def update_image(kernel_size, lower_threshold, upper_threshold, uploaded_image):
    if uploaded_image is not None:
        # Decode the uploaded image and convert it to grayscale
        content_type, content_string = uploaded_image.split(',')
        decoded = base64.b64decode(content_string)
        img_array = np.frombuffer(decoded, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    else:
        return go.Figure()  # Return an empty figure if no image is uploaded

    if img is None:
        return go.Figure()  # If image could not be decoded, return an empty figure

    # Apply Gaussian Blur based on slider input
    kernel_size = kernel_size * 2 + 1  # Ensure odd number for kernel size
    blurred_image = cv2.GaussianBlur(img, (kernel_size, kernel_size), 1.4)

    # Apply Canny Edge Detection with selected thresholds
    edges = cv2.Canny(blurred_image, lower_threshold, upper_threshold)

    # Convert the images to RGB format for visualization
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

    # Create Plotly figure for the images (original + edge detection side by side)
    fig = go.Figure()

    # Add original image on the left
    fig.add_trace(go.Image(z=img_rgb, xaxis='x1', yaxis='y1'))

    # Add edge detection result on the right
    fig.add_trace(go.Image(z=edges_rgb, xaxis='x2', yaxis='y2'))

    fig.update_layout(
        title="Canny Edge Detection",
        xaxis=dict(scaleanchor="y1", showgrid=False, zeroline=False),
        xaxis2=dict(scaleanchor="y2", showgrid=False, zeroline=False),
        yaxis=dict(scaleanchor="x1", showgrid=False, zeroline=False),
        yaxis2=dict(scaleanchor="x2", showgrid=False, zeroline=False),
        grid=dict(rows=1, columns=2),
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [3]:
import base64
import io
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import cv2
import numpy as np
from dash.dependencies import Input, Output
from PIL import Image

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Canny Edge Detection with Non-Maximum Suppression"),
    
    # Image upload
    dcc.Upload(
        id='upload-image',
        children=html.Button('Upload Image'),
        multiple=False
    ),
    
    # Sliders for blur kernel size and Canny thresholds
    dcc.Slider(
        id='kernel-size-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Kernel Size for Blur"),
    
    dcc.Slider(
        id='lower-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=100,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Lower Threshold for Canny Edge Detection"),
    
    dcc.Slider(
        id='upper-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=200,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Upper Threshold for Canny Edge Detection"),
    
    # Graph for displaying the original image and edge detection results
    dcc.Graph(id='edge-detection-graph')
])

# Helper function for Non-Maximum Suppression
def non_maximum_suppression(gradient_magnitude, gradient_direction):
    M, N = gradient_magnitude.shape
    output_image = np.zeros_like(gradient_magnitude, dtype=np.uint8)

    # Quantize the gradient directions into 4 main directions
    angle = gradient_direction * 180. / np.pi
    angle[angle < 0] += 180

    # Loop through each pixel and perform non-maximum suppression
    for i in range(1, M-1):
        for j in range(1, N-1):
            # Get the angle of the gradient
            angle_val = angle[i, j]

            # Horizontal edge (0 degrees)
            if (0 <= angle_val < 22.5) or (157.5 <= angle_val <= 180):
                neighbor1 = gradient_magnitude[i, j+1]
                neighbor2 = gradient_magnitude[i, j-1]
            # 45-degree edge
            elif (22.5 <= angle_val < 67.5):
                neighbor1 = gradient_magnitude[i+1, j-1]
                neighbor2 = gradient_magnitude[i-1, j+1]
            # Vertical edge (90 degrees)
            elif (67.5 <= angle_val < 112.5):
                neighbor1 = gradient_magnitude[i+1, j]
                neighbor2 = gradient_magnitude[i-1, j]
            # 135-degree edge
            else:
                neighbor1 = gradient_magnitude[i-1, j-1]
                neighbor2 = gradient_magnitude[i+1, j+1]

            # Suppress the pixel if it's not the local maximum
            if (gradient_magnitude[i, j] >= neighbor1) and (gradient_magnitude[i, j] >= neighbor2):
                output_image[i, j] = gradient_magnitude[i, j]
            else:
                output_image[i, j] = 0

    return output_image

# Callback to update edge detection results based on sliders
@app.callback(
    Output('edge-detection-graph', 'figure'),
    [Input('kernel-size-slider', 'value'),
     Input('lower-threshold-slider', 'value'),
     Input('upper-threshold-slider', 'value'),
     Input('upload-image', 'contents')]
)
def update_image(kernel_size, lower_threshold, upper_threshold, uploaded_image):
    if uploaded_image is not None:
        # Decode the uploaded image and convert it to grayscale
        content_type, content_string = uploaded_image.split(',')
        decoded = base64.b64decode(content_string)
        img_array = np.frombuffer(decoded, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    else:
        return go.Figure()  # Return an empty figure if no image is uploaded

    if img is None:
        return go.Figure()  # If image could not be decoded, return an empty figure

    # Apply Gaussian Blur based on slider input
    kernel_size = kernel_size * 2 + 1  # Ensure odd number for kernel size
    blurred_image = cv2.GaussianBlur(img, (kernel_size, kernel_size), 1.4)

    # Compute gradient magnitude and direction using Sobel operators
    grad_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    
    gradient_magnitude = np.hypot(grad_x, grad_y)
    gradient_direction = np.arctan2(grad_y, grad_x)

    # Apply Non-Maximum Suppression
    nms_result = non_maximum_suppression(gradient_magnitude, gradient_direction)

    # Apply Canny Edge Detection with selected thresholds (using NMS result)
    edges = cv2.Canny(nms_result, lower_threshold, upper_threshold)

    # Convert the images to RGB format for visualization
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

    # Create Plotly figure for the images (original + edge detection)
    fig = go.Figure()

    # Add original image
    fig.add_trace(go.Image(z=img_rgb))

    # Add edge detection result
    fig.add_trace(go.Image(z=edges_rgb))

    fig.update_layout(title="Canny Edge Detection with Non-Maximum Suppression")

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [4]:
import base64
import io
import dash
from dash import dcc, html  # Import directly from dash
import plotly.graph_objects as go
import cv2
import numpy as np
from dash.dependencies import Input, Output
from PIL import Image

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Canny Edge Detection with Non-Maximum Suppression"),
    
    # Image upload
    dcc.Upload(
        id='upload-image',
        children=html.Button('Upload Image'),
        multiple=False
    ),
    
    # Sliders for blur kernel size and Canny thresholds
    dcc.Slider(
        id='kernel-size-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Kernel Size for Blur"),
    
    dcc.Slider(
        id='lower-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=100,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Lower Threshold for Canny Edge Detection"),
    
    dcc.Slider(
        id='upper-threshold-slider',
        min=0,
        max=255,
        step=1,
        value=200,
        marks={i: str(i) for i in range(0, 256, 50)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Upper Threshold for Canny Edge Detection"),

    # Slider for Non-Maximum Suppression sensitivity
    dcc.Slider(
        id='nms-threshold-slider',
        min=0,
        max=1,
        step=0.01,
        value=0.2,
        marks={i: f'{i:.2f}' for i in np.arange(0, 1.1, 0.2)},
        tooltip={"placement": "bottom", "always_visible": True},
        updatemode='drag'
    ),
    html.Div("Non-Maximum Suppression Sensitivity"),
    
    # Graph for displaying the original image and edge detection results
    dcc.Graph(id='edge-detection-graph')
])

# Helper function for Non-Maximum Suppression
def non_maximum_suppression(gradient_magnitude, gradient_direction, threshold):
    M, N = gradient_magnitude.shape
    output_image = np.zeros_like(gradient_magnitude, dtype=np.uint8)

    # Quantize the gradient directions into 4 main directions
    angle = gradient_direction * 180. / np.pi
    angle[angle < 0] += 180

    # Loop through each pixel and perform non-maximum suppression
    for i in range(1, M-1):
        for j in range(1, N-1):
            # Get the angle of the gradient
            angle_val = angle[i, j]

            # Horizontal edge (0 degrees)
            if (0 <= angle_val < 22.5) or (157.5 <= angle_val <= 180):
                neighbor1 = gradient_magnitude[i, j+1]
                neighbor2 = gradient_magnitude[i, j-1]
            # 45-degree edge
            elif (22.5 <= angle_val < 67.5):
                neighbor1 = gradient_magnitude[i+1, j-1]
                neighbor2 = gradient_magnitude[i-1, j+1]
            # Vertical edge (90 degrees)
            elif (67.5 <= angle_val < 112.5):
                neighbor1 = gradient_magnitude[i+1, j]
                neighbor2 = gradient_magnitude[i-1, j]
            # 135-degree edge
            else:
                neighbor1 = gradient_magnitude[i-1, j-1]
                neighbor2 = gradient_magnitude[i+1, j+1]

            # Suppress the pixel if it's not the local maximum (considering NMS threshold)
            if (gradient_magnitude[i, j] >= neighbor1) and (gradient_magnitude[i, j] >= neighbor2):
                if gradient_magnitude[i, j] >= threshold:  # Apply NMS threshold
                    output_image[i, j] = gradient_magnitude[i, j]
            else:
                output_image[i, j] = 0

    return output_image

# Callback to update edge detection results based on sliders
@app.callback(
    Output('edge-detection-graph', 'figure'),
    [Input('kernel-size-slider', 'value'),
     Input('lower-threshold-slider', 'value'),
     Input('upper-threshold-slider', 'value'),
     Input('nms-threshold-slider', 'value'),
     Input('upload-image', 'contents')]
)
def update_image(kernel_size, lower_threshold, upper_threshold, nms_threshold, uploaded_image):
    if uploaded_image is not None:
        # Decode the uploaded image and convert it to grayscale
        content_type, content_string = uploaded_image.split(',')
        decoded = base64.b64decode(content_string)
        img_array = np.frombuffer(decoded, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    else:
        return go.Figure()  # Return an empty figure if no image is uploaded

    if img is None:
        return go.Figure()  # If image could not be decoded, return an empty figure

    # Apply Gaussian Blur based on slider input
    kernel_size = kernel_size * 2 + 1  # Ensure odd number for kernel size
    blurred_image = cv2.GaussianBlur(img, (kernel_size, kernel_size), 1.4)

    # Compute gradient magnitude and direction using Sobel operators
    grad_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    
    gradient_magnitude = np.hypot(grad_x, grad_y)
    gradient_direction = np.arctan2(grad_y, grad_x)

    # Apply Non-Maximum Suppression with the selected NMS threshold
    nms_result = non_maximum_suppression(gradient_magnitude, gradient_direction, nms_threshold)

    # Apply Canny Edge Detection with selected thresholds (using NMS result)
    edges = cv2.Canny(nms_result, lower_threshold, upper_threshold)

    # Convert the images to RGB format for visualization
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

    # Create Plotly figure for the images (original + edge detection)
    fig = go.Figure()

    # Add original image
    fig.add_trace(go.Image(z=img_rgb))

    # Add edge detection result
    fig.add_trace(go.Image(z=edges_rgb))

    fig.update_layout(title="Canny Edge Detection with Non-Maximum Suppression")

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
